# Abstract

**Author:** [Charles Tapley Hoyt](https://github.com/cthoyt)

This notebook outlines a simple way to explore the citations, authors, and provenance information in a graph and its subgraphs.

### Notebook Setup

In [23]:
import itertools as itt
import os
import time
from collections import defaultdict, Counter
from operator import itemgetter

import pandas as pd
import pybel
import pybel_tools as pbt

from pybel.constants import *

### Notebook Provenance

The time of execution and the versions of the software packegs used are displayed explicitly.

In [24]:
time.asctime()

'Tue Apr 25 11:08:49 2017'

In [25]:
pybel.__version__

'0.5.3'

In [26]:
pbt.__version__

'0.1.8-dev'

### Local Path Definitions

To make this notebook interoperable across many machines, locations to the repositories that contain the data used in this notebook are referenced from the environment, set in `~/.bashrc` to point to the place where the repositories have been cloned. Assuming the repositories have been `git clone`'d into the `~/dev` folder, the entries in `~/.bashrc` should look like:

```bash
...
export BMS_BASE=~/dev/bms
...
```

#### BMS 

The biological model store (BMS) is the internal Fraunhofer SCAI repository for keeping BEL models under version control. It can be downloaded from https://tor-2.scai.fraunhofer.de/gf/project/bms/

In [27]:
bms_base = os.environ['BMS_BASE']

# Data

The Alzheimer's Disease Knowledge Assembly has been precompiled with the following command line script, and will be loaded from this format for improved performance. In general, derived data, such as the gpickle representation of a BEL script, are not saved under version control to ensure that the most up-to-date data is always used.

```sh
pybel convert --path "$BMS_BASE/aetionomy/alzheimers.bel" --pickle "$BMS_BASE/aetionomy/alzheimers.gpickle"
```

The BEL script can also be compiled from inside this notebook with the following python code:

```python
>>> import os
>>> import pybel
>>> # Input from BEL script
>>> bel_path = os.path.join(bms_base, 'aetionomy', 'alzheimers.bel')
>>> graph = pybel.from_path(bel_path)
>>> # Output to gpickle for fast loading later
>>> pickle_path = os.path.join(bms_base, 'aetionomy', 'alzheimers.gpickle')
>>> pybel.to_pickle(graph, pickle_path)
```

In [28]:
pickle_path = os.path.join(bms_base, 'aetionomy', 'alzheimers', 'alzheimers.gpickle')

In [29]:
graph = pybel.from_pickle(pickle_path)

In [30]:
graph.version

'3.0.2'

# Provenance Summary

## Publication Summary

The number of unique referenecs to documents in PubMed can be calculated with [pbt.summary.count_pmids](http://pybel-tools.readthedocs.io/en/latest/summary.html#pybel_tools.summary.count_pmids)

In [31]:
pmid_counter = pbt.summary.count_pmids(graph)

The total number of PubMed references can be readily accessed by the `len()` of the counter output by [pbt.summary.count_pmids](http://pybel-tools.readthedocs.io/en/latest/summary.html#pybel_tools.summary.count_pmids).

In [32]:
len(pmid_counter)

1719

The top 35 most informative papers, in terms of number edges contributed, are displayed below.

In [34]:
pd.DataFrame(pmid_counter.most_common(35), columns='')

,0,1
0,20847424,668
1,22496686,580
2,19885299,418
3,19519303,297
4,19499146,176
5,19734902,168
6,21711233,146
7,24262633,145
8,22110360,132
9,22122372,122


In [33]:
for pmid, count in :
    print('https://www.ncbi.nlm.nih.gov/pubmed/{}\t{}'.format(pmid, count))

https://www.ncbi.nlm.nih.gov/pubmed/20847424	668
https://www.ncbi.nlm.nih.gov/pubmed/22496686	580
https://www.ncbi.nlm.nih.gov/pubmed/19885299	418
https://www.ncbi.nlm.nih.gov/pubmed/19519303	297
https://www.ncbi.nlm.nih.gov/pubmed/19499146	176
https://www.ncbi.nlm.nih.gov/pubmed/19734902	168
https://www.ncbi.nlm.nih.gov/pubmed/21711233	146
https://www.ncbi.nlm.nih.gov/pubmed/24262633	145
https://www.ncbi.nlm.nih.gov/pubmed/22110360	132
https://www.ncbi.nlm.nih.gov/pubmed/22122372	122
https://www.ncbi.nlm.nih.gov/pubmed/22862420	120
https://www.ncbi.nlm.nih.gov/pubmed/18675468	115
https://www.ncbi.nlm.nih.gov/pubmed/19419557	115
https://www.ncbi.nlm.nih.gov/pubmed/19515914	102
https://www.ncbi.nlm.nih.gov/pubmed/24332564	102
https://www.ncbi.nlm.nih.gov/pubmed/14976144	102
https://www.ncbi.nlm.nih.gov/pubmed/10967351	100
https://www.ncbi.nlm.nih.gov/pubmed/25681350	90
https://www.ncbi.nlm.nih.gov/pubmed/22367557	85
https://www.ncbi.nlm.nih.gov/pubmed/22382662	82
https://www.ncbi.nlm.ni

## Citation Enrichment

The NCBI eUtils platform is used to look up all PubMed references and enrich information about the authors, publication, volume, page, and title with [pbt.mutation.fix_pubmed_citations](http://pybel-tools.readthedocs.io/en/latest/mutation.html#pybel_tools.mutation.fix_pubmed_citations).

In [12]:
pbt.mutation.parse_authors(graph)

In [13]:
%%time
erroneous_pmids = pbt.mutation.fix_pubmed_citations(graph, stringify_authors=False)

CPU times: user 648 ms, sys: 59.3 ms, total: 708 ms
Wall time: 30.4 s


### Investigation of Errors

The erroneous PMIDs are summarized below. The evidence strings can be googled to identify the correct publications for recuration of the original BEL document.

In [14]:
pmid_evidences = pbt.summary.get_evidences_by_pmid(graph, erroneous_pmids)

for pmid in sorted(pmid_evidences):
    print('https://www.ncbi.nlm.nih.gov/pubmed/{}'.format(pmid))
    
    for evidence in sorted(pmid_evidences[pmid]):
        print('\t', evidence, '\n')

https://www.ncbi.nlm.nih.gov/pubmed/22377427
	 Here we find that a mitochondrial solute carrier family protein, appoptosin, induces reactive oxygen species release and intrinsic caspase-dependent apoptosis. The physiological function of appoptosin is to transport/exchange glycine/5-amino-levulinic acid across the mitochondrial membrane for heme synthesis. Alzheimer's Abeta-amyloid precursor protein interacts with appoptosin and modulates appoptosin-induced apoptosis. Levels of appoptosin are upregulated in brain samples from Alzheimer's disease and infarct patients and in rodent stroke models, as well as in cells treated with Abeta-amyloid (Abeta) and glutamate. Downregulation of appoptosin prevents the cell death and caspase activation caused by glutamate or Abeta insults. 



## Author Summary

The associations between authors and their publications can be summarized with [pbt.summary.count_author_publications](http://pybel-tools.readthedocs.io/en/latest/summary.html#pybel_tools.summary.count_author_publications).

In [15]:
author_publication_counter = pbt.summary.count_author_publications(graph)

The total number of authors can be readily counted by the `len()` of the Counter returned by [pbt.summary.count_author_publications](http://pybel-tools.readthedocs.io/en/latest/summary.html#pybel_tools.summary.count_author_publications).

In [16]:
len(author_publication_counter)

9219

The top 35 authors, in terms of the number of publications contributed to the graph, are displayed below.

In [17]:
author_publication_counter.most_common(35)

[('Hyman BT', 16),
 ('Li Y', 14),
 ('Wang Y', 14),
 ('Williams J', 14),
 ('Smith MA', 13),
 ('Perry G', 13),
 ('Wang X', 13),
 ('Love S', 12),
 ('Mayeux R', 12),
 ('Farrer LA', 12),
 ('Lovestone S', 12),
 ('Liu Y', 11),
 ('Zhao Y', 11),
 ('Zhang Y', 11),
 ('Haass C', 11),
 ('Younkin SG', 11),
 ('Holtzman DM', 11),
 ('Kehoe PG', 11),
 ('Galimberti D', 11),
 ('Hardy J', 11),
 ('Pericak-Vance MA', 11),
 ('Koo EH', 10),
 ('Mattson MP', 10),
 ('Tan J', 10),
 ("Alzheimer's Disease Neuroimaging Initiative.", 10),
 ('St George-Hyslop P', 10),
 ('Lambert JC', 10),
 ('Amouyel P', 10),
 ('Harold D', 10),
 ('Scarpini E', 10),
 ('Owen MJ', 10),
 ('Petersen RC', 10),
 ('Haines JL', 10),
 ('Xu H', 9),
 ('Zhang J', 9)]

It's also possible to look up the contributions of individual authors using the Counter's dictionary lookup and a simple substring search.

In [18]:
for author in author_publication_counter:
    if 'Heneka' in author:
        print(author, author_publication_counter[author])

Heneka MT 8


The top 35 authors, in terms of the number of edges contributed to the graph, are displayed below.

In [19]:
author_counter = pbt.summary.count_authors(graph)

author_counter.most_common(35)

[('Parihar MS', 677),
 ('Brewer GJ', 668),
 ('Russo C', 623),
 ('Florio T', 623),
 ('Nizzari M', 620),
 ('Pagano A', 583),
 ('Thellung S', 580),
 ('Corsaro A', 580),
 ('Villa V', 580),
 ('Porcile C', 580),
 ('de la Monte SM', 418),
 ('Wands JR', 418),
 ('Hardy J', 363),
 ('Lovestone S', 333),
 ('Williams J', 332),
 ('Younkin SG', 322),
 ('Goate AM', 311),
 ('Hyman BT', 310),
 ('Schubert M', 303),
 ('Cruchaga C', 302),
 ('Carrasquillo MM', 302),
 ('Owen MJ', 300),
 ('Freude S', 297),
 ('Schilbach K', 297),
 ('Harold D', 290),
 ('Kauwe JS', 288),
 ('Kehoe PG', 280),
 ('Sims R', 267),
 ('Morris JC', 267),
 ('Love S', 265),
 ('Gerrish A', 259),
 ('Nowotny P', 258),
 ('Brayne C', 254),
 ('Engelborghs S', 252),
 ('Craig D', 252)]

# Apoptosis Signalling Subgraph Summary

In this example, the Apopotosis Signalling Subgraph is investigated more closely.

In [20]:
target_subgraph = 'Apoptosis signaling subgraph'

In [21]:
subgraph = pbt.selection.get_subgraph_by_annotation_value(graph, annotation='Subgraph', value=target_subgraph)

pbt.summary.print_summary(subgraph)

Number of nodes: 132
Number of edges: 221
Network density: 0.012780476520934536
Number weakly connected components: 10
Average in-degree: 1.6742424242424243
Average out-degree: 1.6742424242424243


The unique citations for every pair of nodes is calculated. This helps to remove the bias from edges that have many notations and have a cartesian explosion. This process can be repeated with [pbt.summary.count_pmids](http://pybel-tools.readthedocs.io/en/latest/summary.html#pybel_tools.summary.count_pmids).

In [22]:
citations = defaultdict(set)

for u, v, d in subgraph.edges_iter(data=True):
    c = d[CITATION]
    citations[u, v].add((c[CITATION_TYPE], c[CITATION_REFERENCE], c[CITATION_NAME]))
    
counter = Counter(itt.chain.from_iterable(citations.values()))

for (_, pmid, name), v in counter.most_common(35):
    print('https://www.ncbi.nlm.nih.gov/pubmed/{}\t{}\t{}' .format(int(pmid.strip()), v, name))

https://www.ncbi.nlm.nih.gov/pubmed/19499146	27	Acta biochimica et biophysica Sinica
https://www.ncbi.nlm.nih.gov/pubmed/22122372	12	Journal of neurochemistry
https://www.ncbi.nlm.nih.gov/pubmed/22496686	11	Journal of toxicology
https://www.ncbi.nlm.nih.gov/pubmed/16153637	9	European journal of pharmacology
https://www.ncbi.nlm.nih.gov/pubmed/17869087	7	The Journal of nutritional biochemistry
https://www.ncbi.nlm.nih.gov/pubmed/19918364	6	PloS one
https://www.ncbi.nlm.nih.gov/pubmed/11592846	6	Neurobiology of disease
https://www.ncbi.nlm.nih.gov/pubmed/12548636	6	Proteomics
https://www.ncbi.nlm.nih.gov/pubmed/19306298	5	Journal of cellular physiology
https://www.ncbi.nlm.nih.gov/pubmed/14744432	5	Cell
https://www.ncbi.nlm.nih.gov/pubmed/18997293	4	Journal of Alzheimer's disease : JAD
https://www.ncbi.nlm.nih.gov/pubmed/22236693	4	Journal of negative results in biomedicine
https://www.ncbi.nlm.nih.gov/pubmed/17316167	4	Current Alzheimer research
https://www.ncbi.nlm.nih.gov/pubmed/19734

# Conclusions

While BEL documents are a repository for biological knowledge, they also provide insight into the most prolific authors and highest information-density papers. After making this information readily available through the functions provided in PyBEL Tools, other tools that handle citation networks could be integrated and utilities like [SCAIView](http://www.scaiview.com/) could be further leveraged to identify which publications would have the highest potential to improve the content of the network.